In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

print(tf.__version__)

2.5.0


In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(0.1 * mnist_info.splits['test'].num_examples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 150

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [3]:
input_size = 784
output_size = 10
hiddent_layer_size = 250

model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(hiddent_layer_size, activation='relu'),
        tf.keras.layers.Dense(hiddent_layer_size, activation='relu'),
        tf.keras.layers.Dense(output_size, activation='softmax'),
    ]
)

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
360/360 - 2s - loss: 0.0117 - accuracy: 0.9959 - val_loss: 0.0139 - val_accuracy: 0.9962
Epoch 2/5
360/360 - 2s - loss: 0.0115 - accuracy: 0.9964 - val_loss: 0.0165 - val_accuracy: 0.9950
Epoch 3/5
360/360 - 2s - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.0126 - val_accuracy: 0.9968
Epoch 4/5
360/360 - 2s - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.0107 - val_accuracy: 0.9968
Epoch 5/5
360/360 - 2s - loss: 0.0121 - accuracy: 0.9958 - val_loss: 0.0144 - val_accuracy: 0.9945


In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 0s 29ms/step - loss: 0.0961 - accuracy: 0.9802


In [9]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 98.02%
